In [1]:
! pip install transformers

In [2]:
import os
import pandas as pd
import json

# Đường dẫn đến thư mục chứa các tệp tin
folder_path = "../data/train/"

# Khởi tạo danh sách để lưu trữ dữ liệu từ mỗi tệp
data_list = []

# Lặp qua tất cả các tệp trong thư mục
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # Chỉ xử lý các tệp có đuôi là .txt
        file_path = os.path.join(folder_path, filename)

        # Đọc dữ liệu từ tệp vào một từ điển Python
        with open(file_path, "r", encoding="utf-8") as file:
            # Giả sử dữ liệu trong tệp có định dạng JSON
            file_data = json.loads(file.read())

        # Thêm dữ liệu từ từ điển vào danh sách
        data_list.append(file_data)

# Tạo DataFrame từ danh sách dữ liệu
df = pd.DataFrame(data_list)

# Tạo hai cột mới cho nội dung và tóm tắt
df["document"] = df["article_text"].apply(lambda x: " ".join(x))
df["summary"] = df["abstract_text"].apply(lambda x: " ".join(x))

# Chọn chỉ một số cột quan trọng nếu cần
df = df[["document", "summary"]]


In [3]:
df

,document,summary
0,redshifts @xmath12.5 witness both the ` quasar...,<S> we report the first detection of the 6.2@x...
1,there is no doubt that galaxies suffer chemica...,<S> we argue that the discrepancies observed i...
2,photonic crystals ( pcs ) describe a class of ...,<S> we investigate the use of a genetic algori...
3,the discovery of new models of quantum computa...,<S> we improve the upper bound on the minimal ...
4,double - logarithmic ( dl ) contributions are ...,<S> it is a brief review on composing and solv...
...,...,...
15995,with the rapid growth of wireless terminals an...,<S> the volume and types of traffic data in mo...
15996,communication is an essential part of the work...,<S> communication in teams is an important but...
15997,since a long time it has been established that...,<S> double parton distribution functions ( dpd...
15998,pattern formation is ubiquitous in spatiotempo...,<S> dynamical patterns in complex networks of ...


In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [9]:
# Tiền xử lý dữ liệu cho mô hình tóm tắt
# Prefix được thêm vào đầu mỗi đoạn văn bản để chỉ định mô hình rằng đây là nhiệm vụ tóm tắt
prefix = "summarize: "

# Hàm tiền xử lý dữ liệu
def preprocess_function(examples):

    documents = examples["document"]
    summaries = examples["summary"]

    # Thêm prefix vào mỗi đoạn văn bản trong cột "document"
    inputs = prefix + documents
    
    # Sử dụng tokenizer để chuyển đổi văn bản thành dạng mà mô hình có thể hiểu
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    
    # Sử dụng tokenizer để chuyển đổi tóm tắt thành dạng mà mô hình có thể hiểu
    labels = tokenizer(text=summaries, max_length=128, truncation=True)
    
    # Gán labels (nhãn) vào model_inputs để sử dụng trong quá trình huấn luyện
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs


In [10]:
tokenized_multi_news =  df.apply(preprocess_function, axis=1)

: 